In [8]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
from nltk.stem import SnowballStemmer

In [9]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('mbi_questions_english.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
# Load the model without compiling it
model = load_model('chatbot_model_eng.keras', compile=False)

In [10]:
def clean_up_sentence(sentence):
    # Convert to lowercase and tokenize the sentence
    sentence_words = nltk.word_tokenize(sentence.lower())
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    # Generate a bag of words for the sentence after cleaning and lowercasing
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    # Use the bag of words model to predict the class with lowercase input
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # Sort results by probability in descending order
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    # Get the response for the top intent
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [11]:
print("GO! Bot is running!")

while True:
    message = input("")
    
    if message.lower() == "good bye":
        print("Chat stopped. Goodbye!")
        break
    
    ints = predict_class(message)
    if ints:  # If an intent is detected
        res = get_response(ints, intents)
    else:
        res = "I don't understand your question, please try again."
    print(res)

GO! Bot is running!
Hi
1/1 [==============================] - 0s 126ms/step
Hello! Let me know if you have any questions.
what is MBI ?
1/1 [==============================] - 0s 33ms/step
The MBI network is a specialized development agency based in Patti, Italy. It is present in 6 countries (United Kingdom, Italy, France, Greece, Romania, and Morocco) and offers a wide range of services. Its areas of expertise include digital marketing, web design, SEO optimization, as well as web and mobile development.
good bye
Chat stopped. Goodbye!


In [12]:
res

'The MBI network is a specialized development agency based in Patti, Italy. It is present in 6 countries (United Kingdom, Italy, France, Greece, Romania, and Morocco) and offers a wide range of services. Its areas of expertise include digital marketing, web design, SEO optimization, as well as web and mobile development.'